In [1]:
import torch
from parksim.intent_predict.cnnV2.network import SimpleCNN, RegularizedCNN, SmallRegularizedCNN
from parksim.intent_predict.cnnV2.utils import CNNDataset, CNNGroupedDataset
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

import os
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
sns.set_theme(style="darkgrid")

In [4]:

def get_predictions(model_path, dji_num):
    model = SmallRegularizedCNN()
    model_state = torch.load(model_path)
    model.load_state_dict(model_state)
    model.eval().cuda()
    dataset = CNNGroupedDataset(f"../data/DJI_{dji_num}", input_transform = transforms.ToTensor())
    dataloader = DataLoader(dataset, batch_size=1, num_workers=12)
    data_size = len(dataset)
    running_top_1_accuracy = 0
    running_top_3_accuracy = 0
    running_top_5_accuracy = 0
    for batch in tqdm(dataloader):
        all_img_features, all_non_spatial_features, all_labels = batch
        for img_feature, non_spatial_feature, labels in zip(all_img_features, all_non_spatial_features, all_labels):
            img_feature = img_feature.cuda()
            non_spatial_feature = non_spatial_feature.cuda()
            labels = labels.cuda()
            num_options = labels.shape[0]
            #model.forward(img_feature, non_spatial_feature)
            #inputs, labels = data[0].to(device), data[1].to(device)

            #optimizer.zero_grad()

            preds = model(img_feature, non_spatial_feature)
            labels = labels.unsqueeze(1)
            label = torch.argmax(labels)
            preds = preds.flatten()
            preds = torch.topk(preds, min(5, num_options))
            pred_indices = preds.indices
            if label in pred_indices[:1]:
                running_top_1_accuracy += 1 / data_size
            if label in pred_indices[:min(3, num_options)]:
                running_top_3_accuracy += 1 / data_size
            if label in pred_indices[:min(5, num_options)]:
                running_top_5_accuracy += 1 / data_size

            #print(label, pred_indices)
            
    print(f"Top 1 Accuracy: {running_top_1_accuracy}")
    print(f"Top 3 Accuracy: {running_top_3_accuracy}")
    print(f"Top 5 Accuracy: {running_top_5_accuracy}")
    return running_top_1_accuracy, running_top_3_accuracy, running_top_5_accuracy

In [5]:
top_1, top_3, top_5 = get_predictions('models/SmallRegularizedIntent-4-10-22.pt', "0021")

100%|██████████| 293/293 [00:01<00:00, 154.43it/s]

Top 1 Accuracy: 0.7542662116040988
Top 3 Accuracy: 1.0000000000000069
Top 5 Accuracy: 1.0000000000000069


In [ ]:
image_features = np.load('../data/DJI_0021_label_grouped.npy', allow_pickle=True)

In [ ]:
all_features = []
for feat in image_features:
    all_features.append(np.array(feat))
all_features = np.array(all_features)
all_features.shape

/tmp/ipykernel_466505/3448905269.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_features = np.array(all_features)


(293,)

In [ ]:
np.save('../data/DJI_0021_label_grouped_test.npy', all_features)

In [ ]:
image_features_test = np.load('../data/DJI_0021_image_feature_grouped_test.npy', allow_pickle=True)

In [ ]:
image_features_test[0].shape

(5, 400, 400, 3)